In [73]:
import boto3
import pandas as pd
from io import BytesIO

In [74]:
# Define S3 bucket and file name
bucket_name = "bnetsacomplaintsdata"
file_name = "Energie_Data_cleaned_short_no_PdB.xlsx"

In [75]:
df = pd.read_excel(file_name)

In [76]:
df.head()

,Unique_Id,Eingangskanal,cleaned_text,Original_Arbeitshilfe_Nummer,Value_Ebene1,Value_Ebene2,Value_Ebene3,Value_Ebene4,Value_Ebene5,Kurztext,Langtext,Code_Nummer_Ebene1,Code_Nummer_Ebene2,Code_Nummer_Ebene3,Code_Nummer_Ebene4,Code_Nummer_Ebene5
0,27442516,email,Jahrgang 2022,VSE-151228130,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN,NaN,Die Preisgestaltung für Strom und Erdgas unter...,1.0,11.0,111.0,NaN,NaN
1,27448449,email,"Sehr geehrte Damen und Herren, Ich habe eine k...",VSE-151228130,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN,NaN,Die Preisgestaltung für Strom und Erdgas unter...,1.0,11.0,111.0,NaN,NaN
2,27448426,email,Von: falk schuster < Email_ersetzt > Gesendet:...,VSE-151228130,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN,NaN,Die Preisgestaltung für Strom und Erdgas unter...,1.0,11.0,111.0,NaN,NaN
3,27445803,email,"Sehr geehrte Damen und Herren, als Kunde der F...",VSE-151228130,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN,NaN,Die Preisgestaltung für Strom und Erdgas unter...,1.0,11.0,111.0,NaN,NaN
4,27445732,email,Hallo vorheriges Jahr wurde mein Vertrag bei E...,VSE-151228130,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN,NaN,Die Preisgestaltung für Strom und Erdgas unter...,1.0,11.0,111.0,NaN,NaN


## Pre-processing data for DBERTA

In [77]:
## Copying dataset in new dataframe 
df_new = df.copy()

In [78]:
df_new.columns

Index(['Unique_Id', 'Eingangskanal', 'cleaned_text',
       'Original_Arbeitshilfe_Nummer', 'Value_Ebene1', 'Value_Ebene2',
       'Value_Ebene3', 'Value_Ebene4', 'Value_Ebene5', 'Kurztext', 'Langtext',
       'Code_Nummer_Ebene1', 'Code_Nummer_Ebene2', 'Code_Nummer_Ebene3',
       'Code_Nummer_Ebene4', 'Code_Nummer_Ebene5'],
      dtype='object')

# Dropping columns 
-Unique_Id -- Unique identifier for each complaint (useful for tracking predictions).
-cleaned_text--Preprocessed complaint text (used as input for DBERTA).
-Value_Ebene1--First-level category (broad category).
-Value_Ebene2--Second-level category (sub-category).
-Value_Ebene3--Third-level category (fine-grained classification).
-Code_Nummer_Ebene1--Numeric representation of first-level category.
-Code_Nummer_Ebene2--Numeric representation of second-level category.
-Code_Nummer_Ebene3--Numeric representation of third-level category.

In [79]:
##Checking categories of each column 
df["Value_Ebene1"].unique()

array(['Strom', 'Sonstiges', nan, 'Gas'], dtype=object)

In [54]:
df["Value_Ebene2"].unique()

array(['Strompreise', nan, 'sonstige Belieferung', 'Irrläufer extern',
       'Rechtsmittel und Handlungsmöglichkeiten', 'Netzanschluss',
       'Abgabe an Dritte', 'Erneuerbare Energien, Kraftwärmekopplung',
       'Messwesen', 'Netzbetrieb', 'Netzregulierung', 'Elektromobilität',
       'Haushaltskundenverträge', 'Lieferantenwechsel', 'Gemischtes',
       'Grundversorgung / Ersatzversorgung', 'Abrechnung', 'Gaspreise',
       'Grundversorgung', 'Netzzugang'], dtype=object)

In [55]:
df["Value_Ebene3"].unique()

array(['keine Strompreisgenehmigung', nan,
       'Kundenanlage § 3 Nr. 24a EnWG',
       'Verbraucherbeschwerde und Schlichtung',
       'Letztverbraucher in Niederspannung, § 18 EnWG',
       'Erneuerbare Energien', 'Befundprüfung von Zählern',
       'Stromnetzentgelte', 'Messstellenbetriebsgesetz',
       'Vertragsschluss', 'Wechselprozess', 'Vor Vertragsschluss',
       'Berater/Projektberatung', 'Grundversorgungsvertragsverhältnis',
       'Im Vertragsverhältnis',
       'Abrechnung & Abrechnungsinformation (Form und Turnus) (§ 40b EnWG)',
       'Verfahren nach 65 EnWG', 'untergeschobene Verträge',
       'Energieträger & Wasser- Unzuständigkeiten',
       'Grundversorgungspflicht', 'Beendigung des Vertrages',
       'Stromabrechnung, Frist § 40c Abs. 2 EnWG',
       'Gasabrechnung, Frist § 40c Abs. 2 EnWG',
       'Beendigung der Grundversorgung', 'Prüfung von Rechnungen',
       'Danksagung vom Verbraucher', 'Gaspreisbremse', 'Heizstrom',
       'Politik & Gesetzgebung- Unzust

In [23]:
df['Value_Ebene4'].unique()

array([nan, 'Kurzform Beschwerde und Schlichtung',
       'Grundstücksbenutzung, § 12 NAV',
       'Abschläge bei Einspeisevergütung',
       'Betrieb elektr. Anlage; Anmeldepflicht Ladesäule; §§ 13, 15, 19 NAV',
       'Nutzung des Anschlusses, § 16 NAV',
       'Unterbrechung  Anschlussnutzung, § 24 Abs. 1 u. 2 NAV – durch Netzbetreiber (aus eigenem Recht)',
       'Messentgelt außerhalb der Verbrauchsrechnung',
       'unerlaubte Telefonwerbung & Textform für Vertragsschluss',
       'Identifizierung der Entnahmestelle, § 14 StromNZV',
       '§ 20a EnWG und Probleme', 'Mini-PV Anlage',
       'Vertragsschluss über Vergleichsportal',
       'Vertragsschluss (§ 2 StromGVV)',
       'Ableserecht von Lieferanten (auch Grundversorgung; § 40a EnWG)',
       'Laufzeit, stillschweigende Vertragsverlängerung, Kündigungsfrist (§ 309 Nr. 9 BGB)',
       'Vertragsbestätigung ohne Vertragsschluss',
       'Umzug (§ 41b Abs. 5 EnWG)', 'Strombezug von PV-Anlagen',
       'Unterbrechung der Versor

In [24]:
df["Value_Ebene5"].unique()

array([nan, 'Preisbindungsklausel', 'Absenkung der EEG-Umlage auf 0',
       'Kündigung durch den Energielieferanten'], dtype=object)

In [25]:
df["Value_Ebene5"].unique()

array([nan, 'Preisbindungsklausel', 'Absenkung der EEG-Umlage auf 0',
       'Kündigung durch den Energielieferanten'], dtype=object)

In [80]:
# List of columns to drop
columns_to_drop = ["Unique_Id","Eingangskanal","Original_Arbeitshilfe_Nummer", 
                   "Kurztext", "Langtext",
                   "Code_Nummer_Ebene1", "Code_Nummer_Ebene2", 
                   "Code_Nummer_Ebene3", "Code_Nummer_Ebene4", 
                   "Code_Nummer_Ebene5"]

# Drop the columns
df_new1 = df_new.drop(columns=columns_to_drop, errors='ignore')

In [81]:
df_new1.columns

Index(['cleaned_text', 'Value_Ebene1', 'Value_Ebene2', 'Value_Ebene3',
       'Value_Ebene4', 'Value_Ebene5'],
      dtype='object')

In [58]:
df_new1.head()

,cleaned_text,Value_Ebene1,Value_Ebene2,Value_Ebene3,Value_Ebene4,Value_Ebene5
0,Jahrgang 2022,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN
1,"Sehr geehrte Damen und Herren, Ich habe eine k...",Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN
2,Von: falk schuster < Email_ersetzt > Gesendet:...,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN
3,"Sehr geehrte Damen und Herren, als Kunde der F...",Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN
4,Hallo vorheriges Jahr wurde mein Vertrag bei E...,Strom,Strompreise,keine Strompreisgenehmigung,NaN,NaN


In [82]:
# Select the first three columns
##Final dataset for the DBERTA model
df_new2 = df_new1.iloc[:, :2]

# Display the first few rows of the new DataFrame
df_new2.head()

,cleaned_text,Value_Ebene1
0,Jahrgang 2022,Strom
1,"Sehr geehrte Damen und Herren, Ich habe eine k...",Strom
2,Von: falk schuster < Email_ersetzt > Gesendet:...,Strom
3,"Sehr geehrte Damen und Herren, als Kunde der F...",Strom
4,Hallo vorheriges Jahr wurde mein Vertrag bei E...,Strom


In [83]:
df_new3 = df_new2.iloc[1:].reset_index(drop=True)

In [84]:
df_new3.head()

,cleaned_text,Value_Ebene1
0,"Sehr geehrte Damen und Herren, Ich habe eine k...",Strom
1,Von: falk schuster < Email_ersetzt > Gesendet:...,Strom
2,"Sehr geehrte Damen und Herren, als Kunde der F...",Strom
3,Hallo vorheriges Jahr wurde mein Vertrag bei E...,Strom
4,"Sehr geehrte Damen und Herren, zu den von mei...",Strom


In [63]:
# Save the DataFrame as a CSV file in SageMaker's local storage
csv_file_path = "/home/ec2-user/SageMaker/preprocessed_data.csv"
df_new3.to_csv(csv_file_path, index=False)

print(f"File saved at: {csv_file_path}")


File saved at: /home/ec2-user/SageMaker/preprocessed_data.csv


In [ ]:
[0.9, 0.12, 0.15]

## 2nd level dataset

In [100]:
df_new4 = df_new1.iloc[:, :3]

In [101]:
df_new4

,cleaned_text,Value_Ebene1,Value_Ebene2
0,Jahrgang 2022,Strom,Strompreise
1,"Sehr geehrte Damen und Herren, Ich habe eine k...",Strom,Strompreise
2,Von: falk schuster < Email_ersetzt > Gesendet:...,Strom,Strompreise
3,"Sehr geehrte Damen und Herren, als Kunde der F...",Strom,Strompreise
4,Hallo vorheriges Jahr wurde mein Vertrag bei E...,Strom,Strompreise
...,...,...,...
717,"PER, habe mit einer Mitarbeiterin von Ihnen te...",Strom,Rechtsmittel und Handlungsmöglichkeiten
718,"Sehr geehrte Damen und Herren, ich hätte eine ...",Strom,NaN
719,"Guten Tag, neue Faxnachricht vom: Datum_erset...",Sonstiges,Irrläufer extern
720,"Guten Morgen, neue Faxnachricht vom: Datum_er...",Sonstiges,Irrläufer extern


In [102]:
df_new4 = df_new4.iloc[1:].reset_index(drop=True)

In [103]:
df_new4.head()

,cleaned_text,Value_Ebene1,Value_Ebene2
0,"Sehr geehrte Damen und Herren, Ich habe eine k...",Strom,Strompreise
1,Von: falk schuster < Email_ersetzt > Gesendet:...,Strom,Strompreise
2,"Sehr geehrte Damen und Herren, als Kunde der F...",Strom,Strompreise
3,Hallo vorheriges Jahr wurde mein Vertrag bei E...,Strom,Strompreise
4,"Sehr geehrte Damen und Herren, zu den von mei...",Strom,Strompreise


In [69]:
df_new4.columns

Index(['cleaned_text', 'Value_Ebene1', 'Value_Ebene2'], dtype='object')

In [104]:
df_new4 = df_new4[['Value_Ebene1', 'cleaned_text', 'Value_Ebene2']]

In [105]:
df_new4

,Value_Ebene1,cleaned_text,Value_Ebene2
0,Strom,"Sehr geehrte Damen und Herren, Ich habe eine k...",Strompreise
1,Strom,Von: falk schuster < Email_ersetzt > Gesendet:...,Strompreise
2,Strom,"Sehr geehrte Damen und Herren, als Kunde der F...",Strompreise
3,Strom,Hallo vorheriges Jahr wurde mein Vertrag bei E...,Strompreise
4,Strom,"Sehr geehrte Damen und Herren, zu den von mei...",Strompreise
...,...,...,...
716,Strom,"PER, habe mit einer Mitarbeiterin von Ihnen te...",Rechtsmittel und Handlungsmöglichkeiten
717,Strom,"Sehr geehrte Damen und Herren, ich hätte eine ...",NaN
718,Sonstiges,"Guten Tag, neue Faxnachricht vom: Datum_erset...",Irrläufer extern
719,Sonstiges,"Guten Morgen, neue Faxnachricht vom: Datum_er...",Irrläufer extern


In [72]:
# Save the DataFrame as a CSV file in SageMaker's local storage
csv_file_path = "/home/ec2-user/SageMaker/preprocessed_data_level2.csv"
df_new4.to_csv(csv_file_path, index=False)

print(f"File saved at: {csv_file_path}")


File saved at: /home/ec2-user/SageMaker/preprocessed_data_level2.csv


In [89]:
## Creating a new column

In [106]:
# Create new column combining LongText + "class - " + Category
df_new4['NewColumn'] = df_new4['cleaned_text'] + " Kategorie ist - " + df_new['Value_Ebene1']

In [107]:
df_new4

,Value_Ebene1,cleaned_text,Value_Ebene2,NewColumn
0,Strom,"Sehr geehrte Damen und Herren, Ich habe eine k...",Strompreise,"Sehr geehrte Damen und Herren, Ich habe eine k..."
1,Strom,Von: falk schuster < Email_ersetzt > Gesendet:...,Strompreise,Von: falk schuster < Email_ersetzt > Gesendet:...
2,Strom,"Sehr geehrte Damen und Herren, als Kunde der F...",Strompreise,"Sehr geehrte Damen und Herren, als Kunde der F..."
3,Strom,Hallo vorheriges Jahr wurde mein Vertrag bei E...,Strompreise,Hallo vorheriges Jahr wurde mein Vertrag bei E...
4,Strom,"Sehr geehrte Damen und Herren, zu den von mei...",Strompreise,"Sehr geehrte Damen und Herren, zu den von mei..."
...,...,...,...,...
716,Strom,"PER, habe mit einer Mitarbeiterin von Ihnen te...",Rechtsmittel und Handlungsmöglichkeiten,"PER, habe mit einer Mitarbeiterin von Ihnen te..."
717,Strom,"Sehr geehrte Damen und Herren, ich hätte eine ...",NaN,"Sehr geehrte Damen und Herren, ich hätte eine ..."
718,Sonstiges,"Guten Tag, neue Faxnachricht vom: Datum_erset...",Irrläufer extern,"Guten Tag, neue Faxnachricht vom: Datum_erset..."
719,Sonstiges,"Guten Morgen, neue Faxnachricht vom: Datum_er...",Irrläufer extern,"Guten Morgen, neue Faxnachricht vom: Datum_er..."


In [108]:

print(df_new4.loc[0, 'NewColumn'])  

Sehr geehrte Damen und Herren, Ich habe eine kurze Frage. Mein Stromanbieter erhöht den Grundpreis zum  Datum_ersetzt  um 19%. (dafür senkt er den Arbeitspreis 😉). Wie oft und in welchem Umfang darf der Stromanbieter den Grundpreis erhöhen ? Gibt es da gesetzliche Grundlagen ? Ich habe im ORG leider nichts gefunden. Vielen Dank. MfG PER Kategorie ist - Strom


In [96]:
df_new4.columns

Index(['cleaned_text', 'Value_Ebene1', 'Value_Ebene2', 'NewColumn'], dtype='object')

In [111]:
df_new5 = df_new4.drop(columns=['cleaned_text', 'Value_Ebene1'])

In [112]:
df_new5

,Value_Ebene2,NewColumn
0,Strompreise,"Sehr geehrte Damen und Herren, Ich habe eine k..."
1,Strompreise,Von: falk schuster < Email_ersetzt > Gesendet:...
2,Strompreise,"Sehr geehrte Damen und Herren, als Kunde der F..."
3,Strompreise,Hallo vorheriges Jahr wurde mein Vertrag bei E...
4,Strompreise,"Sehr geehrte Damen und Herren, zu den von mei..."
...,...,...
716,Rechtsmittel und Handlungsmöglichkeiten,"PER, habe mit einer Mitarbeiterin von Ihnen te..."
717,NaN,"Sehr geehrte Damen und Herren, ich hätte eine ..."
718,Irrläufer extern,"Guten Tag, neue Faxnachricht vom: Datum_erset..."
719,Irrläufer extern,"Guten Morgen, neue Faxnachricht vom: Datum_er..."


In [113]:
df_new5.rename(columns={"NewColumn": "Text"})

,Value_Ebene2,Text
0,Strompreise,"Sehr geehrte Damen und Herren, Ich habe eine k..."
1,Strompreise,Von: falk schuster < Email_ersetzt > Gesendet:...
2,Strompreise,"Sehr geehrte Damen und Herren, als Kunde der F..."
3,Strompreise,Hallo vorheriges Jahr wurde mein Vertrag bei E...
4,Strompreise,"Sehr geehrte Damen und Herren, zu den von mei..."
...,...,...
716,Rechtsmittel und Handlungsmöglichkeiten,"PER, habe mit einer Mitarbeiterin von Ihnen te..."
717,NaN,"Sehr geehrte Damen und Herren, ich hätte eine ..."
718,Irrläufer extern,"Guten Tag, neue Faxnachricht vom: Datum_erset..."
719,Irrläufer extern,"Guten Morgen, neue Faxnachricht vom: Datum_er..."


In [114]:
# Save the DataFrame as a CSV file in SageMaker's local storage
csv_file_path = "/home/ec2-user/SageMaker/preprocessed_data_level3.csv"
df_new5.to_csv(csv_file_path, index=False)

print(f"File saved at: {csv_file_path}")

File saved at: /home/ec2-user/SageMaker/preprocessed_data_level3.csv
